In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

## Load

In [2]:
X, y = np.load('data/X_train.npy'), np.load('data/y_train.npy')
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=.9)

word_to_id = pickle.load(open('data/word_to_id.p', 'rb'))

def get_batches(x, y, batch_size=100):
    n_batches = len(x) // batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for i in range(0, len(x), batch_size):
        yield x[i:i+batch_size], y[i:i+batch_size]

## Hyperparameters

In [3]:
lstm_size = 256
lstm_layers = 3
batch_size = 500
learning_rate = 0.001
embed_size = 300

epochs = 50

checkpoints_dir = 'checkpoints'

os.makedirs(checkpoints_dir, exist_ok=True)

n_words = len(word_to_id)

## Graph

In [4]:
graph = tf.Graph()
with graph.as_default():
    
    # inputs
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # embedding layer
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)
    
    # basic LSTM cell with dropout
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    
    # initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

    # run the data through the RNN nodes
    outputs, final_state = tf.nn.dynamic_rnn(
        cell, embed, initial_state=initial_state)
    
    # name the state tensors
    initial_state = tf.identity(initial_state, 'initial_state')
    final_state = tf.identity(final_state, 'final_state')
    
    # output
    # only care about the final output
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    
    # cost
    cost = tf.losses.mean_squared_error(labels, predictions)
    
    # optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    # accuracy
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    accuracy = tf.identity(accuracy, 'accuracy')

## Training

In [5]:
with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for epoch in range(epochs):
        state = sess.run(initial_state)
        
        for x, y in get_batches(X_train, y_train, batch_size):
            feed = {inputs: x, labels: y[:, None],
                    keep_prob: 0.3, initial_state: state}
            loss, state, _ = sess.run(
                [cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch:", epoch, "Step:", iteration, "Train loss:", loss)

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(X_valid, y_valid, batch_size):
                    feed = {inputs: x, labels: y[:, None], keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Validation accuracy:", np.mean(val_acc))
            iteration +=1
    print('Trining done. Now save.')
    saver.save(sess, checkpoints_dir + '/model.ckpt')

Epoch: 0 Step: 5 Train loss: 0.247256
Epoch: 0 Step: 10 Train loss: 0.23402
Epoch: 0 Step: 15 Train loss: 0.217348
Epoch: 0 Step: 20 Train loss: 0.227762
Epoch: 0 Step: 25 Train loss: 0.205234
Validation accuracy: 0.708
Epoch: 0 Step: 30 Train loss: 0.201352
Epoch: 0 Step: 35 Train loss: 0.211559
Epoch: 0 Step: 40 Train loss: 0.25865
Epoch: 1 Step: 45 Train loss: 0.198156
Epoch: 1 Step: 50 Train loss: 0.231804
Validation accuracy: 0.6925
Epoch: 1 Step: 55 Train loss: 0.209434
Epoch: 1 Step: 60 Train loss: 0.194563
Epoch: 1 Step: 65 Train loss: 0.188647
Epoch: 1 Step: 70 Train loss: 0.188123
Epoch: 1 Step: 75 Train loss: 0.172773
Validation accuracy: 0.761
Epoch: 1 Step: 80 Train loss: 0.182895
Epoch: 2 Step: 85 Train loss: 0.159647
Epoch: 2 Step: 90 Train loss: 0.149752
Epoch: 2 Step: 95 Train loss: 0.14352
Epoch: 2 Step: 100 Train loss: 0.260463
Validation accuracy: 0.771
Epoch: 2 Step: 105 Train loss: 0.179959
Epoch: 2 Step: 110 Train loss: 0.195503
Epoch: 2 Step: 115 Train loss: 0.1

Epoch: 22 Step: 890 Train loss: 0.0148998
Epoch: 22 Step: 895 Train loss: 0.00906328
Epoch: 22 Step: 900 Train loss: 0.0120898
Validation accuracy: 0.839
Epoch: 22 Step: 905 Train loss: 0.0143389
Epoch: 22 Step: 910 Train loss: 0.0197237
Epoch: 22 Step: 915 Train loss: 0.00921261
Epoch: 22 Step: 920 Train loss: 0.0153887
Epoch: 23 Step: 925 Train loss: 0.0239985
Validation accuracy: 0.839
Epoch: 23 Step: 930 Train loss: 0.0160993
Epoch: 23 Step: 935 Train loss: 0.0118446
Epoch: 23 Step: 940 Train loss: 0.0128134
Epoch: 23 Step: 945 Train loss: 0.0138255
Epoch: 23 Step: 950 Train loss: 0.0192076
Validation accuracy: 0.8405
Epoch: 23 Step: 955 Train loss: 0.00879119
Epoch: 23 Step: 960 Train loss: 0.0119819
Epoch: 24 Step: 965 Train loss: 0.024512
Epoch: 24 Step: 970 Train loss: 0.0162766
Epoch: 24 Step: 975 Train loss: 0.00896424
Validation accuracy: 0.8485
Epoch: 24 Step: 980 Train loss: 0.0158276
Epoch: 24 Step: 985 Train loss: 0.0148792
Epoch: 24 Step: 990 Train loss: 0.0199826
Epoch

Epoch: 43 Step: 1735 Train loss: 0.00411853
Epoch: 43 Step: 1740 Train loss: 0.015938
Epoch: 43 Step: 1745 Train loss: 0.0119703
Epoch: 43 Step: 1750 Train loss: 0.0153229
Validation accuracy: 0.8105
Epoch: 43 Step: 1755 Train loss: 0.00601833
Epoch: 43 Step: 1760 Train loss: 0.0127677
Epoch: 44 Step: 1765 Train loss: 0.021016
Epoch: 44 Step: 1770 Train loss: 0.011246
Epoch: 44 Step: 1775 Train loss: 0.0078923
Validation accuracy: 0.842
Epoch: 44 Step: 1780 Train loss: 0.0125027
Epoch: 44 Step: 1785 Train loss: 0.0125273
Epoch: 44 Step: 1790 Train loss: 0.0139702
Epoch: 44 Step: 1795 Train loss: 0.00621055
Epoch: 44 Step: 1800 Train loss: 0.0166453
Validation accuracy: 0.8405
Epoch: 45 Step: 1805 Train loss: 0.0157271
Epoch: 45 Step: 1810 Train loss: 0.0161232
Epoch: 45 Step: 1815 Train loss: 0.00515688
Epoch: 45 Step: 1820 Train loss: 0.0165177
Epoch: 45 Step: 1825 Train loss: 0.012435
Validation accuracy: 0.833
Epoch: 45 Step: 1830 Train loss: 0.0178017
Epoch: 45 Step: 1835 Train los